# Modelo RFM

El modelo RFM (Recency, Frequency, Monetary) es una técnica ampliamente utilizada en análisis de clientes para segmentarlos en función de tres factores:

- **Recency (R)**: Cuánto tiempo ha pasado desde la última compra del cliente.
- **Frequency (F)**: Cuántas compras ha realizado el cliente en un período de tiempo determinado.
- **Monetary (M)**: Cuánto ha gastado el cliente en total.

Para crear el modelo RFM, seguiré los siguientes pasos:

1. **Recency (R)**: Calculará el número de días desde la última compra (fecha de entrega) para cada cliente.
2. **Frequency (F)**: Contar el número de compras (órdenes) de cada cliente.
3. **Monetary (M)**: Sumar el GMV de todas las compras de cada cliente.

Además, conservaré las siguientes características para cada cliente:

- `ontime_num` promedio.
- El último valor de `is_graduated`.
- El último valor de `is_kam`.
- El último valor de `microzona`.
- `birthday`.
- `source_id`.


In [1]:
import pandas as pd

In [2]:
# Cargando el archivo original
df_original = pd.read_csv('data/merged_vlp_otd_anonimized.csv', parse_dates=['deliver_date', 'birthday'])

# Calculando Recency, Frequency, y Monetary para cada cliente usando source_id
rfm_corrected = df_original.groupby('source_id').agg({
    'deliver_date': lambda x: (df_original['deliver_date'].max() - x.max()).days,  # Recency
    'ornum': 'nunique',  # Frequency
    'gmv_on_3': 'sum',  # Monetary
    'ontime_num': 'mean',  # ontime_num promedio
    'is_graduated': 'last',  # último valor de is_graduated
    'is_kam': 'last',  # último valor de is_kam
    'microzone_source_id': 'last',  # último valor de microzone_source_id
    'birthday': 'first'  # birthday
}).reset_index()

# Renombrando las columnas
rfm_corrected.columns = ['customer_id', 'recency', 'frequency', 'monetary', 'ontime_avg', 'is_graduated_last', 'is_kam_last', 'microzona_last', 'birthday']

rfm_corrected.head()


,customer_id,recency,frequency,monetary,ontime_avg,is_graduated_last,is_kam_last,microzona_last,birthday
0,19355299,2,79,9118.95785,0.898734,True,False,33.0,2020-10-15
1,19360755,211,5,531.38090,1.000000,False,False,1188.0,2020-10-16
2,19565513,0,101,7282.90905,0.891089,True,False,1188.0,2020-10-14
3,19566002,76,96,6744.66703,0.916667,False,True,33.0,2020-10-14
4,19567777,6,52,2562.26538,1.000000,True,False,33.0,2020-10-14


In [3]:
# Definiendo funciones para asignar scores basados en percentiles
def r_score(x, percentile_dict):
    """Asignar score para Recency."""
    if x <= percentile_dict[0.2]:
        return 5
    elif x <= percentile_dict[0.4]:
        return 4
    elif x <= percentile_dict[0.6]:
        return 3
    elif x <= percentile_dict[0.8]:
        return 2
    else:
        return 1

def fm_score(x, percentile_dict):
    """Asignar score para Frequency y Monetary."""
    if x <= percentile_dict[0.2]:
        return 1
    elif x <= percentile_dict[0.4]:
        return 2
    elif x <= percentile_dict[0.6]:
        return 3
    elif x <= percentile_dict[0.8]:
        return 4
    else:
        return 5

# Calcular percentiles para Recency, Frequency, y Monetary
percentiles = rfm_corrected[['recency', 'frequency', 'monetary']].quantile(q=[0.2, 0.4, 0.6, 0.8]).to_dict()

# Asignar scores R, F, y M a cada cliente
rfm_corrected['R'] = rfm_corrected['recency'].apply(r_score, args=(percentiles['recency'],))
rfm_corrected['F'] = rfm_corrected['frequency'].apply(fm_score, args=(percentiles['frequency'],))
rfm_corrected['M'] = rfm_corrected['monetary'].apply(fm_score, args=(percentiles['monetary'],))

# Calcular el score RFM total (suma de R, F, y M)
rfm_corrected['RFM_Score'] = rfm_corrected['R'] + rfm_corrected['F'] + rfm_corrected['M']

rfm_corrected[['customer_id', 'R', 'F', 'M', 'RFM_Score']].head()


,customer_id,R,F,M,RFM_Score
0,19355299,4,5,5,14
1,19360755,1,2,2,5
2,19565513,5,5,5,15
3,19566002,2,5,5,12
4,19567777,4,5,4,13


In [4]:
# Uniendo los scores RFM al DataFrame original
df_enriched = df_original.merge(rfm_corrected[['customer_id', 'R', 'F', 'M', 'RFM_Score']], left_on='source_id', right_on='customer_id', how='left')


df_enriched.head()


,source_id,birthday,month_order,year_order,ornum,order_close_date,city,segmento,gmv_on_3,active,...,ontime_num,payment_method_code,dispatch_warehouse,stop_number,customer_id_x,customer_id_y,R,F,M,RFM_Score
0,61675290,2021-09-17,7,2023,61675290_143351339940541,2023-07-28,SPO,Restaurante,32.37042,True,...,True,PIX_ON_DELIVERY,VLP,9,1,61675290,3,2,2,7
1,44578735,2021-04-09,7,2023,44578735_7798210436244,2023-07-28,SPO,Restaurante,84.43154,True,...,True,PIX,VLP,2,2,44578735,5,3,4,12
2,32696268,2021-01-08,7,2023,32696268_4951711420874,2023-07-28,SPO,Restaurante,56.48729,True,...,True,CASH,VLP,10,3,32696268,5,5,4,14
3,135039595,2022-03-31,7,2023,135039595_32785480989704,2023-07-28,SPO,Restaurante,83.47472,True,...,True,PIX_ON_DELIVERY,VLP,2,4,135039595,4,3,3,10
4,228885181,2022-10-07,7,2023,228885181_54824912230348,2023-07-28,SPO,Restaurante,108.02103,True,...,True,CASH,VLP,2,5,228885181,4,4,5,13


Hemos asignado scores RFM a cada cliente:

*R*: Score para Recency. Un valor más alto indica una recencia mayor (es decir, el cliente ha hecho una compra más recientemente).  
*F*: Score para Frequency. Un valor más alto indica una mayor frecuencia de compras.  
*M*: Score para Monetary. Un valor más alto indica un gasto total más alto por parte del cliente  
*RFM_Score*: Es la suma total de los scores R, F y M.  
Con estos scores, podemos comenzar a segmentar a los clientes en diferentes grupos y responder preguntas de negocio relevantes.  

Algunas preguntas de negocio que podríamos abordar incluyen:

## Análisis de Life Time Value (LTV), GMV promedio y frecuencia promedio de compra.

- LTV: Representa el valor neto proyectado de un cliente a lo largo de toda su relación con el negocio. Se calcula multiplicando el GMV promedio por la frecuencia promedio de compra y por el promedio de tiempo que un cliente sigue siendo cliente.
- GMV promedio: Es el ingreso bruto promedio que un cliente genera.
- Frecuencia promedio de compra: Es el número promedio de veces que un cliente hace una compra en un período determinado.

In [5]:
# Calcular la duración promedio de un cliente usando rfm_corrected
rfm_corrected['customer_duration'] = (rfm_corrected['recency'].max() - rfm_corrected['recency']).astype(int)

# Calcular LTV (Life Time Value) para cada cliente
# LTV se calcula como GMV promedio multiplicado por frecuencia promedio de compra
rfm_corrected['LTV'] = rfm_corrected['monetary'] / rfm_corrected['frequency']

# Calculando valores promedio a nivel general
avg_LTV = rfm_corrected['LTV'].mean()
avg_GMV = rfm_corrected['monetary'].mean() / rfm_corrected['frequency'].mean()
avg_frequency = rfm_corrected['frequency'].mean()
avg_customer_duration = rfm_corrected['customer_duration'].mean()

avg_LTV, avg_GMV, avg_frequency, avg_customer_duration


(97.23484108714449, 116.91526394464185, 27.670568360082058, 175.35597924459998)

1. Life Time Value (LTV) Promedio: El LTV promedio es de aproximadamente $97.23. Esto significa que, en promedio, se espera que un cliente aporte $97.23 en valor a lo largo de su tiempo como cliente.
2. GMV Promedio: El valor promedio de GMV por compra es de aproximadamente $116.92.
3. Frecuencia Promedio de Compra: En promedio, un cliente realiza aproximadamente 27.67 compras.
4. Duración Promedio del Cliente: La duración promedio de un cliente (tiempo entre el registro y la última orden) es de aproximadamente 175.36 días.

# Segmentación de Clientes basada en Scores RFM

## Descripción de Segmentos

### 1. Clientes VIP
- **Descripción**: Estos son clientes que han comprado recientemente, compran con frecuencia y gastan mucho.
- **Características**: Alto en Recency, Frequency y Monetary.

### 2. Clientes Leales
- **Descripción**: Estos clientes compran con frecuencia y han gastado una cantidad decente, pero no han comprado tan recientemente como los VIP.
- **Características**: Medio-Alto en Recency, Alto en Frequency y Monetary.

### 3. Clientes en Riesgo
- **Descripción**: Estos clientes solían comprar con frecuencia y gastar, pero ha pasado algún tiempo desde su última compra.
- **Características**: Bajo en Recency, Medio en Frequency y Monetary.

### 4. Clientes Churneados/Zombies
- **Descripción**: Estos son clientes que no han comprado en mucho tiempo, pero en algún momento tuvieron alguna frecuencia de compra y gasto.
- **Características**: Muy bajo en Recency, Bajo en Frequency y Monetary.

## Descubrimientos de los Segmentos

1. **Clientes VIP**: Representan 2.048 clientes. Tienen una recencia baja, una alta frecuencia y un alto valor monetario. 
2. **Clientes Leales**: Representan 1.989 clientes. Su recencia es un poco más alta, pero aún tienen una buena frecuencia y valor monetario.
3. **Clientes en Riesgo**: Representan 2.126 clientes. Tienen una recencia y frecuencia moderadas y un valor monetario intermedio.
4. **Clientes Churneados/Zombies**: Representan 2.124 clientes. No han comprado en mucho tiempo, pero en algún momento tuvieron frecuencia y gasto.

Estos segmentos nos permiten abordar estrategias específicas para cada grupo y entender mejor las características y necesidades de cada segmento.


In [8]:
# Segmentando a los clientes en 4 categorías basadas en el score RFM total
bins = pd.qcut(rfm_corrected['RFM_Score'], q=4, labels=['D', 'C', 'B', 'A'])  # 'A' es la categoría más alta y 'D' la más baja
rfm_corrected['Segment'] = bins

# Conteo de clientes en cada segmento
segment_counts = rfm_corrected['Segment'].value_counts().sort_index()

# Estadísticas descriptivas por segmento
segment_stats = rfm_corrected.groupby('Segment').agg({
    'recency': ['mean', 'median'],
    'frequency': ['mean', 'median'],
    'monetary': ['mean', 'median'],
    'LTV': ['mean', 'median'],
    'customer_duration': ['mean', 'median']
})

segment_counts, segment_stats

# 
# Analizar la relación entre la puntualidad de entrega y los scores RFM

# Calculando el promedio de ontime_num por segmento RFM
ontime_avg_by_segment = rfm_corrected.groupby('Segment')['ontime_avg'].mean()

# Investigar patrones en las microzonas en relación con los scores RFM
microzone_avg_RFM = rfm_corrected.groupby('microzona_last')['RFM_Score'].mean().sort_values(ascending=False)

ontime_avg_by_segment, microzone_avg_RFM.head(10)

# Evaluar la evolución de los scores RFM con el tiempo
# Usaremos la fecha de entrega (deliver_date) como la dimensión temporal

# Calculando el RFM promedio por mes
df_enriched['month_year'] = df_enriched['deliver_date'].dt.to_period('M')
monthly_avg_RFM = df_enriched.groupby('month_year').agg({
    'R': 'mean',
    'F': 'mean',
    'M': 'mean'
}).reset_index()

# Investigar las tendencias de compra alrededor de las fechas de cumpleaños de los clientes
# Creando una columna que indica si la fecha de entrega es el mes de cumpleaños del cliente
df_enriched['birthday_month'] = df_enriched['birthday'].dt.month
df_enriched['is_birthday_month'] = (df_enriched['deliver_date'].dt.month == df_enriched['birthday_month']).astype(int)

# Calculando el porcentaje de órdenes que ocurren en el mes de cumpleaños del cliente
birthday_month_orders = df_enriched['is_birthday_month'].mean()

monthly_avg_RFM, birthday_month_orders



# Cambiando los nombres de los segmentos para que sean más descriptivos
segment_mapping = {
    'A': 'Clientes VIP',
    'B': 'Clientes Leales',
    'C': 'Clientes en Riesgo/Zombies',
    'D': 'Clientes Churneados'
}

rfm_corrected['Segment'] = rfm_corrected['Segment'].map(segment_mapping)

# Mostrando el conteo de clientes en cada nuevo segmento
new_segment_counts = rfm_corrected['Segment'].value_counts().sort_index()

new_segment_counts


Clientes Churneados           2124
Clientes en Riesgo/Zombies    2126
Clientes Leales               1989
Clientes VIP                  2048
Name: Segment, dtype: int64